# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [99]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import myKey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [100]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1750)  
lngs = np.random.uniform(low=-180.000, high=180.000, size=1750)
lat_lngs = zip(lats, lngs)
#uniform is to have the numbers evenly distributed
#size is how many random numbers will be generated.
#we are saying, generate 1500 random evenly distributed numbers between -90 and 90.

# Identify nearest city for each lat, lng combination
for each in lat_lngs:
    city = citipy.nearest_city(each[0], each[1]).city_name 
    country = citipy.nearest_city(each[0], each[1]).country_code 
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
len(cities)

#612 cites are returned.   There must be at least 500 on the next part of finding the weather.  If not, then must come back to this step and increase size.

690

In [101]:
len(countries)

690

In [102]:
#??  how do I see the ln_lngs list?
print(lat_lngs)

In [103]:
print(lngs)

[ 173.99989125   59.33216417  168.51459808 ... -164.93624352   20.14143801
  166.60192619]


In [104]:
#

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [105]:
#first, perform a check on a single city.

In [106]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "alofi"
units = "imperial"

# Build query URL
query_url = url + "appid=" + myKey + "&q=" + city + "&units=imperial"
#note:   adding imperial units will return the temperature in farenheight and the wind speed in miles per hour
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&q=alofi&units=imperial


In [107]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()


In [108]:
print(json.dumps(weather_json, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 68
    },
    "cod": 200,
    "coord": {
        "lat": -19.06,
        "lon": -169.92
    },
    "dt": 1554066000,
    "id": 4036284,
    "main": {
        "humidity": 94,
        "pressure": 1011,
        "temp": 77,
        "temp_max": 77,
        "temp_min": 77
    },
    "name": "Alofi",
    "sys": {
        "country": "NU",
        "id": 7306,
        "message": 0.0069,
        "sunrise": 1554053160,
        "sunset": 1554096080,
        "type": 1
    },
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 230,
        "speed": 12.75
    }
}


In [109]:
#Success!   Able to retrieve a single city.
#now retrieve each individual piece of data needed for the loop

In [110]:
print(weather_json["name"])

Alofi


In [111]:
print(weather_json["coord"]["lat"])

-19.06


In [112]:
print(weather_json["main"]["temp"])

77


In [113]:
print(weather_json["main"]["humidity"])

94


In [114]:
print(weather_json["clouds"]["all"])

68


In [115]:
print(weather_json["wind"]["speed"])

12.75


### need a dataframe containing city and country to begin
NOTE:   the Weather API will take only a city; however, if multiples are found, API responds with a list of results that match a searching word. 
<BR>
Therefore, I will begin with a list of cities and countries which can also be passed to the 
    <BR>q city name and country code divided by comma, use ISO 3166 country codes
        
<BR> https://openweathermap.org/current

In [116]:
#Begin with the city and country list which was created in the begining.

cityCountry = pd.DataFrame(list(zip(cities, countries)),
              columns=['city','country'])

cityCountry.head()

,city,country
0,lolua,tv
1,taolanaro,mg
2,pevek,ru
3,ushuaia,ar
4,punta arenas,cl


In [117]:
#confirm there are at least 500 countries
len(cityCountry)

690

In [118]:
# create a data frome with the values needed for the API calls

cityCountry2 = cityCountry
cityCountry2["location"] = (cityCountry2["city"] + ", " + cityCountry2["country"])
cityCountry2["newCity"] = ""
cityCountry2["newCountry"] = ""
cityCountry2["lat"] = ""
cityCountry2["long"] = ""
cityCountry2["tempF"] = ""
cityCountry2["humidity"] = ""
cityCountry2["cloud"] = ""
cityCountry2["windMPH"] = ""
cityCountry2["maxTemp"] = ""
cityCountry2.head()

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,lolua,tv,"lolua, tv",,,,,,,,,
1,taolanaro,mg,"taolanaro, mg",,,,,,,,,
2,pevek,ru,"pevek, ru",,,,,,,,,
3,ushuaia,ar,"ushuaia, ar",,,,,,,,,
4,punta arenas,cl,"punta arenas, cl",,,,,,,,,


## Retrieve Data from the API
Practice on 10 cities before attemping to get data for 500+ cities

In [119]:
#create a new df with only 10 records
cityCountrySample = cityCountry2.head(10)
cityCountrySample.head(20)

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,lolua,tv,"lolua, tv",,,,,,,,,
1,taolanaro,mg,"taolanaro, mg",,,,,,,,,
2,pevek,ru,"pevek, ru",,,,,,,,,
3,ushuaia,ar,"ushuaia, ar",,,,,,,,,
4,punta arenas,cl,"punta arenas, cl",,,,,,,,,
5,sovetskiy,ru,"sovetskiy, ru",,,,,,,,,
6,bluff,nz,"bluff, nz",,,,,,,,,
7,chokurdakh,ru,"chokurdakh, ru",,,,,,,,,
8,rikitea,pf,"rikitea, pf",,,,,,,,,
9,new norfolk,au,"new norfolk, au",,,,,,,,,


In [120]:
#ensure I can get a single response using parameters
parameters = {"appid": myKey,
             "units": "imperial",
             "q": "vaini, to"}
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [121]:
#get the API response
singleResponse = requests.get(base_url, params=parameters)

In [122]:
#confirm the URL
print(singleResponse.url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=vaini%2C+to


In [123]:
#convert the response to JSON 
singleResponseJ = singleResponse.json()

In [124]:
#print the response to view the data
print(json.dumps(singleResponseJ, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 75
    },
    "cod": 200,
    "coord": {
        "lat": -21.2,
        "lon": -175.2
    },
    "dt": 1554062400,
    "id": 4032243,
    "main": {
        "humidity": 88,
        "pressure": 1011,
        "temp": 80.6,
        "temp_max": 80.6,
        "temp_min": 80.6
    },
    "name": "Vaini",
    "sys": {
        "country": "TO",
        "id": 7285,
        "message": 0.004,
        "sunrise": 1554140885,
        "sunset": 1554183652,
        "type": 1
    },
    "visibility": 10000,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 130,
        "speed": 9.17
    }
}


In [125]:
#success!  I can get a valid URL using parameters.  

In [126]:
#Adding to ignore the warning for appending the values in the next step.  Error is: 
#pd.options.mode.chained_assignment = None  # default='warn'   

###  Complete for 10 Records
Before applying to all records, complete the for loop for a small sample.

In [127]:
# create a parameters dict that will be updated with new city each iteration
#per API, imperial units will obtain temperature in F and wind speed in mph
parameters = {"appid": myKey,
             "units": "imperial"}

print("---------------------------------------------") 
print("Beginning Data Retrieval")
print("---------------------------------------------") 
      
#begin by knowing the number of cities
print("Number of cities to obtain data: " + str(len(cityCountry))) 
print("---------------------------------------------")       

# Loop through the cityCountrySample and perform a search on each
for index, row in cityCountrySample.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #print the record number  ??? how do I print the index number?
    print("Retrieving index # " + str(index) + " | " + row["city"] + ", " + row["country"])
    
    #confirm the URL
    print("  " + (cities_data.url))
        
    # convert to json
    citiesJ = cities_data.json()
    
    try:
        cityCountrySample.loc[index, "newCity"] = citiesJ["name"]
    except:
        print("     Missing city name... skipping data.")     
    
    try:    
        cityCountrySample.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    except: 
        print("     Missing country name... skipping data.")
 
    try:    
        cityCountrySample.loc[index, "lat"] = citiesJ["coord"]["lat"]
    except: 
        print("     Missing latitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "long"] = citiesJ["coord"]["lon"]
    except: 
        print("     Missing longitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "tempF"] = citiesJ["main"]["temp"]
    except: 
        print("     Missing temp(F)... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "humidity"] = citiesJ["main"]["humidity"]
    except: 
        print("     Missing humidity... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "cloud"] = citiesJ["clouds"]["all"]
    except: 
        print("     Missing %of cloudiness... skipping data.")
     
    try:    
        cityCountrySample.loc[index, "windMPH"] = citiesJ["wind"]["speed"]
    except: 
        print("     Missing wind speed... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "maxTemp"] = citiesJ["main"]["temp_max"]
    except: 
        print("     Missing maximum temperature... skipping data.")
    
    #the openweather API is free for up to 60 calls per minute.  Add a sleep timer to wait .75 seconds between calls.
    time.sleep(.75)

   

#At least 500 cities must be included for the next step of creating graphics.   Ensure there are at least 500 cities.

#obtain the number of records where a city was not found
noRecord = len(cityCountrySample[cityCountrySample['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

#subtract these from the number of records at the beginning
endRecords = (len(cityCountry)) - noRecord

initRecords = (len(cityCountry))

print("---------------------------------------------")
print("Beginning Records: " + str(initRecords))
print("Ending Records: " + str(endRecords))

if endRecords < 500:
    print("The data sample is less than 500, which is too small.  Please start from the beginning to obtain a new data sample.")
    print("---------------------------------------------")   
    #store the dataframe to a csv so the user can get details if needed.
    cityCountrySample.to_csv("weatherData_sample10.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData_sample10.csv")
    print("Date Retrieval Complete")
    print("---------------------------------------------")   

else: 
    print("There are more than 500 ending records!   Continue to the graphs.")


print("---------------------------------------------")   
print("Date Retrieval Complete")
print("---------------------------------------------")   

#store the datframe to a csv
noRecord = len(apiData[apiData['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))noRecord = len(apiData[apiData['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

# Visualize to confirm new city and new country appear
cityCountrySample.head(10)


---------------------------------------------
Beginning Data Retrieval
---------------------------------------------
Number of cities to obtain data: 690
---------------------------------------------
Retrieving index # 0 | lolua, tv
  http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=lolua%2Ctv
     Missing city name... skipping data.
     Missing country name... skipping data.
     Missing latitude... skipping data.
     Missing longitude... skipping data.
     Missing temp(F)... skipping data.
     Missing humidity... skipping data.
     Missing %of cloudiness... skipping data.
     Missing wind speed... skipping data.
     Missing maximum temperature... skipping data.
Retrieving index # 1 | taolanaro, mg
  http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=taolanaro%2Cmg
     Missing city name... skipping data.
     Missing country name... skipping data.
     Missing latitude... 

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,lolua,tv,"lolua, tv",,,,,,,,,
1,taolanaro,mg,"taolanaro, mg",,,,,,,,,
2,pevek,ru,"pevek, ru",Pevek,RU,69.7,170.27,-0.26,77,0,3.69,-0.26
3,ushuaia,ar,"ushuaia, ar",Ushuaia,AR,-54.81,-68.31,51.04,81,75,16.11,51.8
4,punta arenas,cl,"punta arenas, cl",Punta Arenas,CL,-53.16,-70.91,55.24,66,75,13.87,55.4
5,sovetskiy,ru,"sovetskiy, ru",Sovetskiy,RU,56.76,48.47,34.49,88,0,14.65,34.49
6,bluff,nz,"bluff, nz",Bluff,NZ,-46.6,168.33,55,73,48,10,55
7,chokurdakh,ru,"chokurdakh, ru",Chokurdakh,RU,70.62,147.9,9.65,70,36,2.91,9.65
8,rikitea,pf,"rikitea, pf",Rikitea,PF,-23.12,-134.97,76.88,100,68,18.57,76.88
9,new norfolk,au,"new norfolk, au",New Norfolk,AU,-42.78,147.06,46.42,66,75,9.17,50


## Retrieve data on all cities 
Since the practice run on 10 entries works well, complete the exercise on all records.

In [128]:
#create a new df to store the values from the API
apiData = cityCountry2
apiData.head(20)

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,lolua,tv,"lolua, tv",,,,,,,,,
1,taolanaro,mg,"taolanaro, mg",,,,,,,,,
2,pevek,ru,"pevek, ru",,,,,,,,,
3,ushuaia,ar,"ushuaia, ar",,,,,,,,,
4,punta arenas,cl,"punta arenas, cl",,,,,,,,,
5,sovetskiy,ru,"sovetskiy, ru",,,,,,,,,
6,bluff,nz,"bluff, nz",,,,,,,,,
7,chokurdakh,ru,"chokurdakh, ru",,,,,,,,,
8,rikitea,pf,"rikitea, pf",,,,,,,,,
9,new norfolk,au,"new norfolk, au",,,,,,,,,


In [ ]:
# create a parameters dict that will be updated with new city each iteration
#per API, imperial units will obtain temperature in F and wind speed in mph
parameters = {"appid": myKey,
             "units": "imperial"}

print("---------------------------------------------") 
print("Beginning Data Retrieval")
print("---------------------------------------------") 
      
#begin by knowing the number of cities
print("Number of cities to obtain data: " + str(len(cityCountry))) 
print("---------------------------------------------")       

# Loop through the cityCountrySample and perform a search on each
for index, row in apiData.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #print the record number  ??? how do I print the index number?
    print("Retrieving index # " + str(index) + " | " + row["city"] + ", " + row["country"])
    
    #confirm the URL
    print("  " + (cities_data.url))
        
    # convert to json
    citiesJ = cities_data.json()
    
    try:
        apiData.loc[index, "newCity"] = citiesJ["name"]
    except:
        print("     Missing city name... skipping data.")     
    
    try:    
        apiData.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    except: 
        print("     Missing country name... skipping data.")
 
    try:    
        apiData.loc[index, "lat"] = citiesJ["coord"]["lat"]
    except: 
        print("     Missing latitude... skipping data.")
        
    try:    
        apiData.loc[index, "long"] = citiesJ["coord"]["lon"]
    except: 
        print("     Missing longitude... skipping data.")
        
    try:    
        apiData.loc[index, "tempF"] = citiesJ["main"]["temp"]
    except: 
        print("     Missing temp(F)... skipping data.")
        
    try:    
        apiData.loc[index, "humidity"] = citiesJ["main"]["humidity"]
    except: 
        print("     Missing humidity... skipping data.")
        
    try:    
        apiData.loc[index, "cloud"] = citiesJ["clouds"]["all"]
    except: 
        print("     Missing %of cloudiness... skipping data.")
     
    try:    
        apiData.loc[index, "windMPH"] = citiesJ["wind"]["speed"]
    except: 
        print("     Missing wind speed... skipping data.")
        
    try:    
        apiData.loc[index, "maxTemp"] = citiesJ["main"]["temp_max"]
    except: 
        print("     Missing maximum temperature... skipping data.")
    
    #the openweather API is free for up to 60 calls per minute.  Add a sleep timer to wait .75 seconds between calls.
    time.sleep(.75)

   
#At least 500 cities must be included for the next step of creating graphics.   Ensure there are at least 500 cities.

#obtain the number of records where a city was not found
noRecord = len(apiData[apiData['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

#subtract these from the number of records at the beginning
endRecords = (len(cityCountry)) - noRecord

initRecords = (len(cityCountry))

print("---------------------------------------------")
print("Beginning Records: " + str(initRecords))
print("Ending Records: " + str(endRecords))

if endRecords < 500:
    print("The data sample is less than 500, which is too small.  Please start from the beginning to obtain a new data sample.")
    print("---------------------------------------------")  
    
    #save the data to a csv in case the user needs to view the details
    apiData.to_csv("weatherData_TooSmall.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData_TooSmall.csv")
    
    print("Date Retrieval Complete")
    print("---------------------------------------------")   

else: 
    print("There are more than 500 ending records!   Continue to the graphs.")
    apiData.to_csv("weatherData.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData.csv")
    


print("---------------------------------------------")   
print("Date Retrieval Complete")
print("---------------------------------------------")   
    
# Visualize to confirm new city and new country appear
apiData.head(10)


In [130]:
noRecord = len(apiData[apiData['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

Records not found in the weather database: 85


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot